## Análisis de comportamiento de usuarios en una tienda en línea de gadgets tecnológicos

Objetivo: Analizar el comportamiento de los usuarios en un sitio de e-commerce de productos tecnológicos (celulares, audífonos, smartwatches, accesorios) para identificar patrones de compra, embudos de conversión y recomendaciones para mejorar ventas.

In [81]:
# Importamos las librerías
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import scipy.stats as st
from datetime import datetime
import math as mth

In [82]:
# Leémos los datos
users = pd.read_csv("datasets/users.csv")
sessions = pd.read_csv("datasets/sessions.csv")
orders = pd.read_csv("datasets/orders.csv")
product_views = pd.read_csv("datasets/product_views.csv")

### Preparación de los datos

#### Users:

In [83]:
#Revisamos la información general
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   user_id         120 non-null    object
 1   edad            120 non-null    int64 
 2   genero          120 non-null    object
 3   pais            120 non-null    object
 4   fecha_registro  120 non-null    object
dtypes: int64(1), object(4)
memory usage: 4.8+ KB


In [84]:
#Revisamos los datos númericos generales
users.describe()

,edad
count,120.000000
mean,38.050000
std,12.444735
min,18.000000
25%,26.000000
50%,39.000000
75%,50.000000
max,59.000000


In [85]:
#Impresión rápida de nuestros datos para analizar el contenido
users.sample(5)

,user_id,edad,genero,pais,fecha_registro
95,U096,59,M,Perú,2024-01-01
33,U034,24,F,México,2024-11-16
101,U102,25,F,Argentina,2024-02-17
82,U083,24,F,Chile,2024-11-14
53,U054,21,M,Chile,2024-10-31


In [86]:
#Revisamos los valores vacíos
users.isna().sum()

user_id           0
edad              0
genero            0
pais              0
fecha_registro    0
dtype: int64

In [87]:
#Revisamos si tenemos datos duplicados
duplicates = users.duplicated().sum()
print('Valores duplicados: ',duplicates)

Valores duplicados:  0


In [88]:
#Corregimos el tipo de dato de object a datetime
users['fecha_registro'] = pd.to_datetime(users['fecha_registro'])

In [89]:
#Confirmamos que todos los cambios se hayan realizado
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   user_id         120 non-null    object        
 1   edad            120 non-null    int64         
 2   genero          120 non-null    object        
 3   pais            120 non-null    object        
 4   fecha_registro  120 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 4.8+ KB


#### Sessions:

In [90]:
#Revisamos la información general
sessions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   session_id         350 non-null    object
 1   user_id            350 non-null    object
 2   fecha_sesion       350 non-null    object
 3   canal              350 non-null    object
 4   duracion_segundos  350 non-null    int64 
 5   paginas_vistas     350 non-null    int64 
dtypes: int64(2), object(4)
memory usage: 16.5+ KB


In [91]:
#Revisamos los datos númericos generales
sessions.describe()

,duracion_segundos,paginas_vistas
count,350.000000,350.000000
mean,334.365714,4.962857
std,160.017273,2.608405
min,62.000000,1.000000
25%,200.250000,3.000000
50%,344.000000,5.000000
75%,460.750000,7.000000
max,598.000000,9.000000


In [92]:
#Impresión rápida de nuestros datos para analizar el contenido
sessions.sample(5)

,session_id,user_id,fecha_sesion,canal,duracion_segundos,paginas_vistas
310,S311,U033,2024-04-07,orgánico,221,8
306,S307,U118,2024-04-08,email,446,6
21,S022,U100,2024-05-12,email,157,6
146,S147,U083,2024-05-27,orgánico,439,4
257,S258,U085,2024-03-02,redes,281,8


In [93]:
#Revisamos los valores vacíos
sessions.isna().sum()

session_id           0
user_id              0
fecha_sesion         0
canal                0
duracion_segundos    0
paginas_vistas       0
dtype: int64

In [94]:
#Revisamos si tenemos datos duplicados
duplicates = sessions.duplicated().sum()
print('Valores duplicados: ',duplicates)

Valores duplicados:  0


In [95]:
#Corregimos el tipo de dato de object a datetime
sessions['fecha_sesion'] = pd.to_datetime(sessions['fecha_sesion'])

In [96]:
#Confirmamos que todos los cambios se hayan realizado
sessions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   session_id         350 non-null    object        
 1   user_id            350 non-null    object        
 2   fecha_sesion       350 non-null    datetime64[ns]
 3   canal              350 non-null    object        
 4   duracion_segundos  350 non-null    int64         
 5   paginas_vistas     350 non-null    int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 16.5+ KB


#### Orders:

In [97]:
#Revisamos la información general
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   order_id      180 non-null    object 
 1   user_id       180 non-null    object 
 2   fecha_compra  180 non-null    object 
 3   monto         180 non-null    float64
 4   categoria     180 non-null    object 
 5   metodo_pago   180 non-null    object 
dtypes: float64(1), object(5)
memory usage: 8.6+ KB


In [98]:
#Revisamos los datos númericos generales
orders.describe()

,monto
count,180.000000
mean,1354.221444
std,643.095884
min,301.260000
25%,793.512500
50%,1342.750000
75%,1904.412500
max,2494.580000


In [99]:
#Impresión rápida de nuestros datos para analizar el contenido
orders.sample(5)

,order_id,user_id,fecha_compra,monto,categoria,metodo_pago
173,O174,U078,2024-04-20,2026.16,smartphone,tarjeta
169,O170,U030,2024-05-11,591.09,laptop,tarjeta
98,O099,U003,2024-05-18,2436.37,smartwatch,transferencia
105,O106,U104,2024-05-04,1710.12,smartphone,transferencia
25,O026,U042,2024-03-06,1040.51,laptop,tarjeta


In [100]:
#Revisamos los valores vacíos
orders.isna().sum()

order_id        0
user_id         0
fecha_compra    0
monto           0
categoria       0
metodo_pago     0
dtype: int64

In [101]:
#Revisamos si tenemos datos duplicados
duplicates = orders.duplicated().sum()
print('Valores duplicados: ',duplicates)

Valores duplicados:  0


In [102]:
#Corregimos el tipo de dato de object a datetime
orders['fecha_compra'] = pd.to_datetime(orders['fecha_compra'])

In [103]:
#Confirmamos que todos los cambios se hayan realizado
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   order_id      180 non-null    object        
 1   user_id       180 non-null    object        
 2   fecha_compra  180 non-null    datetime64[ns]
 3   monto         180 non-null    float64       
 4   categoria     180 non-null    object        
 5   metodo_pago   180 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 8.6+ KB


#### Product views:

In [104]:
#Revisamos la información general
product_views.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_id           400 non-null    object 
 1   fecha             400 non-null    object 
 2   producto_id       400 non-null    object 
 3   categoria         400 non-null    object 
 4   precio            400 non-null    float64
 5   agregado_carrito  400 non-null    bool   
dtypes: bool(1), float64(1), object(4)
memory usage: 16.1+ KB


In [105]:
#Revisamos los datos númericos generales
product_views.describe()

,precio
count,400.000000
mean,1427.527850
std,640.281764
min,300.890000
25%,874.780000
50%,1448.490000
75%,1977.110000
max,2499.800000


In [106]:
#Impresión rápida de nuestros datos para analizar el contenido
product_views.sample(5)

,user_id,fecha,producto_id,categoria,precio,agregado_carrito
225,U020,2024-04-10,P003,smartphone,673.32,False
60,U086,2024-05-15,P013,smartwatch,1222.29,False
3,U091,2024-03-08,P005,smartphone,516.52,False
320,U100,2024-05-07,P006,smartphone,1668.97,False
166,U084,2024-04-10,P010,laptop,2427.16,False


In [107]:
#Revisamos los valores vacíos
product_views.isna().sum()

user_id             0
fecha               0
producto_id         0
categoria           0
precio              0
agregado_carrito    0
dtype: int64

In [108]:
#Revisamos si tenemos datos duplicados
duplicates = product_views.duplicated().sum()
print('Valores duplicados: ',duplicates)

Valores duplicados:  0


In [109]:
#Corregimos el tipo de dato de object a datetime
product_views['fecha'] = pd.to_datetime(product_views['fecha'])

In [111]:
#Confirmamos que todos los cambios se hayan realizado
product_views .info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   user_id           400 non-null    object        
 1   fecha             400 non-null    datetime64[ns]
 2   producto_id       400 non-null    object        
 3   categoria         400 non-null    object        
 4   precio            400 non-null    float64       
 5   agregado_carrito  400 non-null    bool          
dtypes: bool(1), datetime64[ns](1), float64(1), object(3)
memory usage: 16.1+ KB


### Análisis exploratorio (EDA)

In [140]:
# Distribución de usuarios por país y edad.
users_dist = users.groupby(['pais']).agg(edad_min=('edad','min'),edad_max=('edad','max'),edad_mean=('edad','mean'),edad_median=('edad','median')).reset_index()
users_dist

,pais,edad_min,edad_max,edad_mean,edad_median
0,Argentina,18,59,38.900000,40.5
1,Chile,18,57,38.233333,40.0
2,Colombia,19,46,31.416667,29.5
3,México,18,59,37.800000,37.5
4,Perú,20,59,41.166667,43.0


In [115]:
# Categorías más vendidas.
sales_category = orders.groupby('categoria').agg(ventas_total=('monto','sum')).reset_index().sort_values('ventas_total', ascending=False) 
sales_category

,categoria,ventas_total
1,laptop,72365.00
0,accesorios,59511.62
2,smartphone,57369.55
3,smartwatch,54513.69


In [142]:
# Canales de adquisición con más conversiones.

#Vamos a unir el dato del sessions con los clientes
conversiones_merge = pd.merge(sessions,orders, on='user_id', how='outer')
conversiones_filtered = conversiones_merge[conversiones_merge['order_id'].notnull()]
conversiones_filtered
conversiones_group = conversiones_merge.groupby('canal').agg(usuarios_unicos_total=('user_id','nunique'),monto_total=('monto','sum')).reset_index().sort_values('usuarios_unicos_total', ascending=False) 
conversiones_group

,canal,usuarios_unicos_total,monto_total
3,redes,64,212267.80
0,anuncios,63,161324.88
2,orgánico,63,134445.74
1,email,58,159985.71


In [164]:
# Tiempo promedio entre registro y primera compra.
first_date_order = orders.groupby('user_id')['fecha_compra'].min().reset_index()


users_first_sale = pd.merge(users,first_date_order, on='user_id', how='left')
users_first_sale['diff_days'] = users_first_sale['fecha_compra'] - users_first_sale['fecha_registro']

users_filter = users_first_sale[users_first_sale['diff_days']> '0 days'].sort_values('diff_days', ascending=False)
#first_date_order
#users_first_sale['diff_days'].mean()
users_filter['diff_days'].mean()


Timedelta('38 days 22:51:25.714285714')

In [139]:
# Tasa de conversión: usuarios con sesión vs usuarios con compra.
orders_u = orders['user_id'].nunique()
sessions_u = sessions['user_id'].nunique()
print(f"Usuarios únicos con compras: {orders_u}")
print(f"Usuarios únicos con sesiones: {sessions_u}")
print(f"Conversión del {(orders_u/sessions_u):.2%}")

Usuarios únicos con compras: 88
Usuarios únicos con sesiones: 113
Conversión del 77.88%
